Loading the CSV file and creating a target

In [18]:
import pandas as pd
from sqlalchemy import create_engine

# Load dataset
df = pd.read_csv("flight_data_2024.csv")

# ✅ Use lowercase column name 'arr_delay'
df["Delayed"] = (df["arr_delay"] > 15).astype(int)

# Keep relevant columns
df = df[[
    "year", "month", "day_of_week", "fl_date",
    "op_unique_carrier", "op_carrier_fl_num",
    "origin", "dest", "dep_delay", "arr_delay",
    "distance", "cancelled", "diverted", "Delayed"
]]

# Save to DB
engine = create_engine("sqlite:///flights2024.db")
df.to_sql("flights2024", con=engine, if_exists="replace", index=False)

print("✅ flights2024 table created successfully")



C:\Users\swara_mmys6c\AppData\Local\Temp\ipykernel_6296\1834188584.py:5: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("flight_data_2024.csv")


✅ flights2024 table created successfully


Quering and preprocessing

In [19]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

query = """
SELECT year, month, day_of_week, dep_delay, distance,
       op_unique_carrier, origin, dest, cancelled, diverted, Delayed
FROM flights2024
WHERE cancelled = 0 AND diverted = 0
"""
df = pd.read_sql(query, engine)

# Encode categorical columns
for col in ["op_unique_carrier", "origin", "dest"]:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop("Delayed", axis=1)
y = df["Delayed"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Scale numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Training the model

In [20]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


✅ Accuracy: 0.9326932050013854
              precision    recall  f1-score   support

           0       0.94      0.98      0.96   1112376
           1       0.92      0.73      0.81    280678

    accuracy                           0.93   1393054
   macro avg       0.93      0.86      0.89   1393054
weighted avg       0.93      0.93      0.93   1393054



Save Predictions

In [21]:
# Predict on full dataset
X_full = scaler.transform(X)
df["Predicted"] = model.predict(X_full)

df.to_sql("flight_preds_2024", con=engine, if_exists="replace", index=False)

print("✅ Predictions saved to flight_preds_2024 table")

✅ Predictions saved to flight_preds_2024 table
